# 패키지 임포트

In [1]:
import folium
import json
import matplotlib.pyplot as plt
import pandas as pd

# CSV 파일 로드

In [2]:
df = pd.read_csv("C://Users/user/Desktop/project/3.csv", encoding='UTF-8')
df4 = pd.read_csv("C://Users/user/Desktop/project/4.csv", encoding='UTF-8')
df5 = pd.read_csv("C://Users/user/Desktop/project/5.csv", encoding='UTF-8')
df6 = pd.read_csv("C://Users/user/Desktop/project/6.csv", encoding='UTF-8')
df7 = pd.read_csv("C://Users/user/Desktop/project/7.csv", encoding='UTF-8')
df8 = pd.read_csv("C://Users/user/Desktop/project/8.csv", encoding='UTF-8')
df9 = pd.read_csv("C://Users/user/Desktop/project/9.csv", encoding='UTF-8')
df10 = pd.read_csv("C://Users/user/Desktop/project/10.csv", encoding='UTF-8')

In [3]:
code = pd.read_csv("data/admin_code.csv", encoding='UTF-8')
code = code.rename(columns = {'행자부행정동코드':'행정동코드'})

# 각 테이블을 통합

In [4]:
df = pd.concat([df, df4, df5, df6, df7, df8, df9, df10])

# 전체 구에 대한 테이블 정리

In [5]:
df_all = df[['기준일ID','시간대구분','행정동코드','총생활인구수']]
df_all_dong = df_all.groupby('행정동코드').mean()[['총생활인구수']]
df_ad = pd.merge(df_all_dong, code, on = '행정동코드')[['시군구명','행정동명','행정동코드','총생활인구수']]
df_ad.head()

,시군구명,행정동명,행정동코드,총생활인구수
0,종로구,청운효자동,11110515,17230.058686
1,종로구,사직동,11110530,34423.790553
2,종로구,삼청동,11110540,7363.778302
3,종로구,부암동,11110550,13720.575727
4,종로구,평창동,11110560,20220.681235


# 전체 구 지도에 필요한 정보 설정

In [36]:
geo_path = 'C://Users/user/anaconda_src/data/dong_shape.json'
geo_str = json.load(open(geo_path, encoding='utf-8-sig'))
guDat = pd.DataFrame({'adm_nm':df_ad['행정동명'], 'counts':df_ad['총생활인구수']})
guDat.sort_values(by = 'counts', ascending = False).head()

,adm_nm,counts
367,역삼1동,112974.654613
287,여의동,94692.125964
214,서교동,85365.836005
8,종로1.2.3.4가동,76875.596338
195,신촌동,70205.678107


# 전체 구 지도

In [56]:
map = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)
map.choropleth(geo_data=geo_str,
                  data=guDat,
                  columns=['adm_nm', 'counts'],
                  key_on='feature.properties.adm_nm',
                  fill_color='YlGnBu',
                  line_color='black')
map

# 누적 하위 5% 구만 필터링해 새로운 테이블 생성

In [8]:
df_add = df_ad.set_index("시군구명")
df_ad5 = df_add.loc[["노원구","중랑구","성동구","강북구","강서구","광진구","동작구","서대문구","강동구","관악구","용산구","동대문구","성북구","도봉구","구로구"]][["행정동명","총생활인구수"]]
df_add.reset_index()
df_ad5.sort_values(by = "총생활인구수", ascending = False).head()

,행정동명,총생활인구수
시군구명,,
서대문구,신촌동,70205.678107
강동구,길동,54386.260288
노원구,공릉2동,52623.664866
동작구,노량진1동,50439.488565
강서구,화곡1동,49467.481633


# 누적 하위 5% 이하 구 지도에 필요한 정보 설정

In [9]:
geo_path = 'C://Users/user/anaconda_src/data/dong_shape.json'
geo_str = json.load(open(geo_path, encoding='utf-8-sig'))
guDat5 = pd.DataFrame({'adm_nm':df_ad5['행정동명'], 'counts':df_ad5['총생활인구수']})
guDat5.head()

,adm_nm,counts
시군구명,,
노원구,월계1동,25353.125329
노원구,월계2동,20279.761603
노원구,월계3동,27221.970921
노원구,공릉1동,43458.543123
노원구,공릉2동,52623.664866


In [26]:
map = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11, tiles='stamentoner')
map.choropleth(geo_data=geo_str,
                  data=guDat5,
                  columns=['adm_nm', 'counts'],
                  fill_color='YlGnBu', #PuRd, YlGnBu
                  key_on='feature.properties.adm_nm')
map

# 원하는 구의 이름과 인구수만 필터링

In [28]:
dfs = df_add.loc['중구'].reset_index()[['행정동명','총생활인구수']]

In [29]:
geo_path = 'C://Users/user/anaconda_src/data/dong_shape.json'
geo_str = json.load(open(geo_path, encoding='utf-8-sig'))
guDatS = pd.DataFrame({'adm_nm':dfs['행정동명'], 'counts':dfs['총생활인구수']})
guDatS

,adm_nm,counts
0,소공동,32701.242149
1,회현동,29919.262783
2,명동,45330.617930
3,필동,17907.826743
4,장충동,13500.428804
5,광희동,24135.283608
6,을지로동,18083.931198
7,신당동,27497.204014
8,다산동,16832.743880
9,약수동,15978.696575


In [67]:
map = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11, tiles='stamentoner')
map.choropleth(geo_data=geo_str,
                  data=guDatS,
                  columns=['adm_nm', 'counts'],
                  fill_color='YlGnBu', #PuRd, YlGnBu
                  key_on='feature.properties.adm_nm')
map

In [73]:
mar_path = 'C://Users/user/anaconda_src/data/Market.geojson'
mar_str = json.load(open(mar_path, encoding='utf-8-sig'))

In [74]:
folium.GeoJson(mar_str, name='json_data').add_to(map)

In [75]:
map

In [ ]:
map_osm = folium.Map(location=[37.566345, 126.977893])

rfile = open(‘d:/temp/chicken_data/skorea_provinces_geo_simple.json’, ‘r’, encoding=’utf-8′).read()